[Spotify API Search](https://developer.spotify.com/documentation/web-api/reference/search/search/)

In [2]:
# !pip install spotipy

In [2]:
from __future__ import print_function    # (at top of module)
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import time
import sys
import pandas as pd
import numpy as np
import re

In [5]:
cid = <id>
secret = <secret>
username = <username>
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)     
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
results = sp.search(q='kanye', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print(' ', i, t['name'])

  0 Mixed Personalities (feat. Kanye West)
  1 Kanye
  2 I Love It (& Lil Pump)
  3 Gold Digger
  4 POWER
  5 All Mine
  6 Heartless
  7 MAMA (feat. Nicki Minaj, Kanye West)
  8 Stronger
  9 Father Stretch My Hands Pt. 1
  10 Waves
  11 Ghost Town
  12 All Of The Lights
  13 Black Skinhead
  14 Famous
  15 Watch (feat. Lil Uzi Vert & Kanye West)
  16 Yikes
  17 Ultralight Beam
  18 Runaway
  19 Violent Crimes


In [52]:
def users_to_playlists(users):
    playlists = []
    for user in users:
        try:
            for item in sp.user_playlists(user)['items']:
                try:
                    playlists.append({'user_id':user, 'playlist_id':item['id'], 'added_by_id':item['owner']['id']})
                except:
                    print('except playlist')
        except:
            print('except user')
        print('user_complete')
    return playlists


In [49]:
def parse_playlist_data(userID, playlistID, limit = 100, fields = 'items,total,next'):
    '''
    Used to get information about the tracks in a user's spotify playlist
    
    Arguments:zz
    userID is a spotify user ID
    playlistID is a spotify playlist ID
    limit is the number of tracks to retrieve in each call. "Next" is used to get all songs. 100 is the max to
        retrieve at once
    fields is the fields to retrieve for each playlist call
    
    Returns:
    A list of dictionaries with each dictionary corresponding to a song.
    The keys of this dictionary are: user_id, playlist_id, added_by_id, album_name,
                album_id, artist_names, artist_ids, track_name, track_id, date_added
    '''
    playlist = sp.user_playlist_tracks(userID,
                            playlist_id=playlistID,
                            fields = fields,
                            limit = limit)
    test_data_list = []
    for i in range(len(playlist['items'])):
        artist_name_list = []
        artist_id_list = []
        added_by_id = playlist['items'][i]['added_by']['id']
        added_at = playlist['items'][i]['added_at']
        curr = playlist['items'][i]['track']
        for j in range(len(curr['artists'])):
            artist_name_list.append(curr['artists'][j]['name'])
            artist_id_list.append(curr['artists'][j]['id'])
        track_name = curr['name']
        track_id = curr['id']
        album_name = curr['album']['name']
        album_id = curr['album']['id']
        test_data_list.append({'user_id': userID,
                               'playlist_id': playlistID,
                               'added_by_id': added_by_id,
                               'album_name': album_name,
                               'album_id': album_id,
                               'artist_names':artist_name_list,
                               'artist_ids':artist_id_list,
                               'track_name': track_name,
                               'track_id': track_id,
                               'date_added': added_at
                              })
    # While next exists, more pages of results exist, so the loop continues until all data is retrieved
    while playlist['next']:
        playlist = sp.next(playlist)
        for i in range(len(playlist['items'])):
            artist_name_list = []
            artist_id_list = []
            added_by_id = playlist['items'][i]['added_by']['id']
            added_at = playlist['items'][i]['added_at']
            curr = playlist['items'][i]['track']
            for j in range(len(curr['artists'])):
                artist_name_list.append(curr['artists'][j]['name'])
                artist_id_list.append(curr['artists'][j]['id'])
            track_name = curr['name']
            track_id = curr['id']
            album_name = curr['album']['name']
            album_id = curr['album']['id']
            test_data_list.append({'user_id': userID,
                                   'playlist_id': playlistID,
                                   'added_by_id': added_by_id,
                                   'album_name': album_name,
                                   'album_id': album_id,
                                   'artist_names':artist_name_list,
                                   'artist_ids':artist_id_list,
                                   'track_name': track_name,
                                   'track_id': track_id,
                                   'date_added': added_at
                                  })
    return test_data_list

In [72]:
pd.DataFrame(test_data_list)

,album_id,album_name,artist_ids,artist_names,track_id,track_name
0,5ErQ2Er5dxb26cG52Kizfj,"Øfdream: Anthology, Pt. 1",[70P3LvwslDBQicUfL2Ix02],[Øfdream],098PU9y0wA153H6UI36uMy,Thelema
1,65zyTIOImGO6U6gkn6Aacr,Dead to Me,"[5No9X0ZejwFDnD1dVllwQA, 3QryVD03gGZOLQQXjy3EoA]","[Terror Reid, Getter]",26zo23frbrONsEJs6m2drZ,Dead to Me
2,6FpMYELyYcOYbyusvv2nVz,Try Me,[0DHLXYQowVvc8oQxp5Lrk3],[foxwedding],2FhGitc3KC7g8VBQDzKW6E,Try Me
3,4wwAaOFLUUkdPORispyWGA,Elysian Fields,[6F6njae9EOpXLsIXP9SiWM],[CVRL],3V3EweEH3Tpq29nyzRey36,Subterfuge
4,3iBo6Zw5NbDfTDzddQ4OKb,Night's Edge,[4F4w1Gkfja6wPJzuMKCLmk],[Noah B],6LJajfVgTG4q9HrL6BHTWd,Contact
5,6IJNG7N2Dbzcdbm9QwZ9va,Hypersquare,"[4ufkY8hmhmYl4aCnzv3dLE, 5puIJpEBWYS37qD8TS8gZq]","[Deadcrow, MYSTXRIVL]",5pDFJgkTXczsJIIDspl8hF,Hypersquare
6,4fPasVJUT0Wp9skLi6EP3R,Plastician Presents - Wavepool 2,[6YoGAC3Z4jQdZHFlGu8Hlo],[CLFRD],6NCLQheJJDquZfIXMlhZn8,Expansion
7,4fPasVJUT0Wp9skLi6EP3R,Plastician Presents - Wavepool 2,[4F4w1Gkfja6wPJzuMKCLmk],[Noah B],1UsHLvLnGBCYWJflJWIFbp,Nylon
8,5dm1oq4zHFfFHEITmYrNcv,Love Dream Infinite,[5JQljNQvDpk1hhKpJHpwNU],[Klimeks],7rGI2jhDcnnfSJXXTpGDCV,Transcend
9,5aTvGoBADTe9XQVwPvq4b9,Atlantis,"[53Wk6hhoFwz3REXaZ2XW2a, 7y4breKexfBWgdyMgHMEHK]","[Pholo, Kareful]",0tvsr8pIUKtq1QkVJ0DxM6,Atlantis


In [25]:
# load in list of spotify usernames found with reddit api
np.load('first_users_spotifyplaylists.npy')

array(['1143975465', 'webbstur', 'spotify', 'crow74', '1234856699',
       'joyegwong', '1236033088', 'z73500if87uo2s5dmla6dmw80',
       'dominicwolfmusic', 'dadacoi', 'issa461',
       '2p1bf9t77iet1qrhtmwjpx4fl', 'havenyates',
       'cosj7zcap1o7p99uvu0moic8v', '9mys2mq0uyiqmce2ufesmvi5n', 'uoak',
       'lg7gkjmevh6brhby94ofq7a4m', '12102288431',
       '3142mwop3oevrd3xaik7mhz7oia4', 'hilltopchill', 'open_heavens',
       'dylan.kelly02', 'lefuturewave', '3lmdcjyfcxuys41prmy8raj7v',
       '8o3mdwilp2ll6m5m6tes4ajro', 'domirillo', 'speny7',
       're2moa6b4alnbtyt1042ks32k', 'warddeblonde',
       '22b2riu4prpktds3fmqx5esri', '1187411126', 'zerochances',
       'janne.kynnas', 'momoofficial1995', 'jrvfln4tdrj21d236bb0cpgsr',
       'marina.resende02', 'laurenmar1e', 'r6q0tjjylq3c6ceo9wot8zz8g',
       'silverclaw82', 'tunenewmusic', 'kimjoh88',
       'go71sqdu4tlp97bhtiu7d0prq', 'connormahone-us',
       'cw26o0cja1jjps3vtc8js0bmd', '12154946292', 'stoneygatesound',
       '0sj

Note: Spotify API retrieves tracks that are no longer available.

In [3]:
# read in spotify user ids found with the reddit api
df = pd.read_json('user_ids_fri.json')

In [45]:
users = list(df[0])

In [53]:
playlists = users_to_playlists(users)

user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_complete
user_c

In [56]:
df_playlists = pd.DataFrame(playlists)

In [61]:
df_playlists.to_json('playlists_fri.json')

In [67]:
df_playlists.head()

,added_by_id,playlist_id,user_id
0,brendan.ta,3pTPXB3vT93AOTSVozP54o,brendan.ta
1,brendan.ta,095Q1U6Hyk9J8zgcBLXZSV,brendan.ta
2,brendan.ta,0W42jQAELrPbNTF2u2vqgv,brendan.ta
3,brendan.ta,09IWLqL8IAXijc2Oa4oXM6,brendan.ta
4,brendan.ta,1BZXb715L2PTSNxjT1Ed4W,brendan.ta


In [60]:
df_playlists[df_playlists['added_by_id']=='spotify']

,added_by_id,playlist_id,user_id
85,spotify,37i9dQZF1DWYkaDif7Ztbp,1243463050
231,spotify,37i9dQZF1CyOz7XTwMpU00,connormahone-us
263,spotify,37i9dQZF1CyPVSZJFAFmHW,sageguitar70
440,spotify,37i9dQZF1DX5hR0J49CmXC,1279525315
441,spotify,37i9dQZF1DX4JAvHpjipBk,1279525315
442,spotify,37i9dQZF1DX0XUsuxWHRQd,1279525315
482,spotify,37i9dQZF1DX3PFzdbtx1Us,zerochances
538,spotify,37i9dQZF1CyP65zkbR7T9Q,tdstone
551,spotify,37i9dQZF1DX9myttyycIxA,tdstone
638,spotify,37i9dQZF1DWWYlokKRoQoC,kingdong9


In [68]:
playlist_curr = list(df_playlists.playlist_id)
user_curr = list(df_playlists.user_id)
feed = list(zip(user_curr, playlist_curr))

In [69]:
feed

[('brendan.ta', '3pTPXB3vT93AOTSVozP54o'),
 ('brendan.ta', '095Q1U6Hyk9J8zgcBLXZSV'),
 ('brendan.ta', '0W42jQAELrPbNTF2u2vqgv'),
 ('brendan.ta', '09IWLqL8IAXijc2Oa4oXM6'),
 ('brendan.ta', '1BZXb715L2PTSNxjT1Ed4W'),
 ('brendan.ta', '1Tr7X2y3FKie5f1CJlYEQf'),
 ('brendan.ta', '1FrKvPLnIjQwRxBZtXm2Hs'),
 ('brendan.ta', '3LF6iEL6uCE9rSdLF4m1ga'),
 ('brendan.ta', '3yXbwr81hcRLCx0JEb567o'),
 ('brendan.ta', '4UiYc3lcKixaZe5eFrpVIR'),
 ('brendan.ta', '0WrbMIFKW83c0Whl5o9Zxd'),
 ('brendan.ta', '6DBYdTePXqjCYLH8YuYpiV'),
 ('brendan.ta', '3fk9YREC123mLq2VtGNINC'),
 ('brendan.ta', '0qVfV2SozMW4pSMvdXmD13'),
 ('brendan.ta', '3By1s2KdULO5w5JxqRvW8R'),
 ('brendan.ta', '0a8L35wbXEBD1l4Mc9vSm3'),
 ('brendan.ta', '2JUQhTbzfm6aEE5L29a5h0'),
 ('brendan.ta', '53nS1D5vs7H5OMNO3GyWmk'),
 ('brendan.ta', '5OMbPQXcHlr9HHrrxdqfwi'),
 ('brendan.ta', '0WZIG5YuV3ep66xY9EjCQb'),
 ('brendan.ta', '329aiPISvEvfQpuBH0KWqW'),
 ('brendan.ta', '2IN1jWkpBy6j4dpoMvqzIO'),
 ('brendan.ta', '01VpWWi8xzzS6nJ64J5hQA'),
 ('brendan.

In [87]:
all_data_test.shape
new_list = sum(all_data_test,[])

In [88]:
new_list

[{'user_id': 'brendan.ta',
  'playlist_id': '3pTPXB3vT93AOTSVozP54o',
  'added_by_id': 'brendan.ta',
  'album_name': 'What Went Down',
  'album_id': '4uIDigk79DeZEYV6Z5Yf4s',
  'artist_names': ['Foals'],
  'artist_ids': ['6FQqZYVfTNQ1pCqfkwVFEa'],
  'track_name': 'Mountain At My Gates',
  'track_id': '53L6A3I9vf7rgEZnMzx54E',
  'date_added': '2016-03-26T00:48:16Z'},
 {'user_id': 'brendan.ta',
  'playlist_id': '3pTPXB3vT93AOTSVozP54o',
  'added_by_id': 'brendan.ta',
  'album_name': 'In The Silence (Deluxe Version)',
  'album_id': '4sFhah3DYcJlYeT47q3rhM',
  'artist_names': ['Ásgeir'],
  'artist_ids': ['7xUZ4069zcyBM4Bn10NQ1c'],
  'track_name': 'King and Cross',
  'track_id': '6VNo09sojPBi5mdckQkLbX',
  'date_added': '2016-03-26T00:49:53Z'},
 {'user_id': 'brendan.ta',
  'playlist_id': '3pTPXB3vT93AOTSVozP54o',
  'added_by_id': 'brendan.ta',
  'album_name': 'ZABA',
  'album_id': '7jfkffCqKYEJmIgGOIJ9me',
  'artist_names': ['Glass Animals'],
  'artist_ids': ['4yvcSjfu4PC0CYQyLy4wSq'],
  't

In [89]:
df_all_test = pd.DataFrame(new_list)

In [90]:
df_all_test.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta
2,brendan.ta,7jfkffCqKYEJmIgGOIJ9me,ZABA,[4yvcSjfu4PC0CYQyLy4wSq],[Glass Animals],2016-03-26T00:49:59Z,3pTPXB3vT93AOTSVozP54o,3djK2yDqJArNAzHyhzSf8G,Gooey,brendan.ta
3,brendan.ta,1VwxJUHMuoppsVjf3VWcZb,Swim,[4aEnNH9PuU1HF3TsZTru54],[Caribou],2016-03-26T00:52:14Z,3pTPXB3vT93AOTSVozP54o,5uBOzETXaVhfUvo0a0X1TZ,Odessa,brendan.ta
4,brendan.ta,1vJRG3hs7MpA7wVwrXoNWJ,Our Love,[4aEnNH9PuU1HF3TsZTru54],[Caribou],2016-03-26T00:52:22Z,3pTPXB3vT93AOTSVozP54o,23FA91vzMAs3k0gMV3ZGaV,Can't Do Without You,brendan.ta


In [91]:
# df_all_test.to_json('fri_working_data.json')

In [3]:
df_all_test = pd.read_json('fri_working_data.json')

In [6]:
df_all_test[df_all_test.playlist_id == '002MBUoZVJntSIsgW6EpFA']

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id
170818,bballgolfer,5gditTVu3JNE5P8T6tafiH,20th Century Masters The Millennium Collection...,[3zXw2Eh96iTT51pytzHdZi],[38 Special],2019-03-14T06:33:44Z,002MBUoZVJntSIsgW6EpFA,6hQszXJIYhtjmSTmDqZpeU,Hold On Loosely,bballgolfer
170819,bballgolfer,4vWxauP6oFowwSQeBnFwRW,Special Forces,[3zXw2Eh96iTT51pytzHdZi],[38 Special],2019-03-14T06:33:51Z,002MBUoZVJntSIsgW6EpFA,3jYRpwbctfqB77uU7T7K3U,Caught Up In You,bballgolfer
170820,bballgolfer,19lEZSnCCbVEkKchoPQWDZ,Aerosmith,[7Ey4PD4MYsKc5I2dolUwbH],[Aerosmith],2019-03-14T06:34:14Z,002MBUoZVJntSIsgW6EpFA,5MxNLUsfh7uzROypsoO5qe,Dream On,bballgolfer
170821,bballgolfer,616UgaZTc454Gu0aXxVH3K,I Don't Want To Miss A Thing,[7Ey4PD4MYsKc5I2dolUwbH],[Aerosmith],2019-03-14T06:34:17Z,002MBUoZVJntSIsgW6EpFA,0c1gHntWjKD7QShC8s99sq,I Don't Want to Miss a Thing - From the Touchs...,bballgolfer
170822,bballgolfer,3XYqOJI1YlX40kJTdzFEzp,Permanent Vacation,[7Ey4PD4MYsKc5I2dolUwbH],[Aerosmith],2019-03-14T06:34:22Z,002MBUoZVJntSIsgW6EpFA,3kfXUGIdBNpyr2gBvU3Guj,Angel,bballgolfer
170823,bballgolfer,36IxIOGEBAXVozDSiVs09B,Toys In The Attic,[7Ey4PD4MYsKc5I2dolUwbH],[Aerosmith],2019-03-14T06:34:27Z,002MBUoZVJntSIsgW6EpFA,24NwBd5vZ2CK8VOQVnqdxr,Sweet Emotion,bballgolfer
170824,bballgolfer,1Z0JbC9RPMJNxE82vTQcIX,All Out Of Love,[4xXCRXOfQKQ2gjWxNhNzYW],[Air Supply],2019-03-14T06:34:33Z,002MBUoZVJntSIsgW6EpFA,01MXkFA8sL7at6txavDErt,Making Love Out of Nothing at All,bballgolfer
170825,bballgolfer,2Rn1tjI9H6vZp9ZlxAZKce,American Pride,[6rJqqRce0Kvo2dJUXoHleC],[Alabama],2019-03-14T06:34:48Z,002MBUoZVJntSIsgW6EpFA,1N9GGOUf8TQqsaPDByZzEF,I'm in a Hurry (And Don't Know Why),bballgolfer
170826,bballgolfer,0eMs6LyA1bTZm8Mx8FPshM,Closer You Get,[6rJqqRce0Kvo2dJUXoHleC],[Alabama],2019-03-14T06:34:59Z,002MBUoZVJntSIsgW6EpFA,0bM1z18RQpr61UuI1LemIK,Dixieland Delight,bballgolfer
170827,bballgolfer,1DE97Q1jJABKEbOi8CzLSu,The Essential Alabama,[6rJqqRce0Kvo2dJUXoHleC],[Alabama],2019-03-14T06:35:03Z,002MBUoZVJntSIsgW6EpFA,0HgpiaXip83f5BARVLfvu2,Keepin' Up,bballgolfer


In [7]:
df_all_test.shape

(751157, 10)

In [93]:
df_all_test.track_id.nunique()

395441

In [5]:
track_counts = df_all_test.track_id.value_counts()

In [104]:
track_counts

0bYg9bo50gSsH3LtXe2SQn    140
2xLMifQCjDGFmkHkpNLD9h    116
2JvzF1RMd7lE3KmFlsyZD8     97
2374M0fQpWi3dLnB54qaLX     95
40riOy7x9W7GXjyGp4pjAv     94
7sO5G9EABYOXQKNPNiE9NR     89
7w87IxuO7BDcJ3YUqCyMTT     87
2dpaYNEQHiRxtZbfNsse99     87
2IRZnDFmlqMuOrYOLnZZyc     86
7ycWLEP1GsNjVvcjawXz3z     86
2Fxmhks0bxGSBdJ92vM42m     85
7m9OqQk4RVRkw9JJdeAw96     85
7dt6x5M1jzdTEt8oCbisTK     85
4qKcDkK6siZ7Jp1Jb4m0aL     83
6gBFPUFcJLzWGx4lenP6h2     78
08mG3Y1vljYA6bvDt4Wqkj     74
1LzNfuep1bnAUR9skqdHCK     74
0I3q5fE6wg7LIfHGngUTnV     71
1jJci4qxiYcOHhQR247rEU     71
2aoo2jlRnM3A0NyLQqMN2f     71
0rTV5WefWd1J3OwIheTzxM     71
6MWtB6iiXyIwun0YzU6DFP     71
5p7ujcrUXASCNwRaWNHR1C     70
6L89mwZXSOwYl76YXfX13s     69
7COXchtUOMd6uIT6HvmRaI     68
6u7jPi22kF8CTQ3rb9DHE7     68
0d28khcov6AiegSCpG5TuT     68
0b9oOr2ZgvyQu88wzixux9     67
3FtYbEfBqAlGO46NUDQSAt     66
6Qs4SXO9dwPj5GKvVOv8Ki     66
                         ... 
2oCIHudP9wgI3OshvOtNfG      1
75K1Vpg7BhP3xC256aqtKv      1
53aUYPTwJe

In [19]:
frequent_tracks.index

Index(['0bYg9bo50gSsH3LtXe2SQn', '2xLMifQCjDGFmkHkpNLD9h',
       '2JvzF1RMd7lE3KmFlsyZD8', '2374M0fQpWi3dLnB54qaLX',
       '40riOy7x9W7GXjyGp4pjAv', '7sO5G9EABYOXQKNPNiE9NR',
       '7w87IxuO7BDcJ3YUqCyMTT', '2dpaYNEQHiRxtZbfNsse99',
       '7ycWLEP1GsNjVvcjawXz3z', '2IRZnDFmlqMuOrYOLnZZyc',
       '7m9OqQk4RVRkw9JJdeAw96', '2Fxmhks0bxGSBdJ92vM42m',
       '7dt6x5M1jzdTEt8oCbisTK', '4qKcDkK6siZ7Jp1Jb4m0aL',
       '6gBFPUFcJLzWGx4lenP6h2', '1LzNfuep1bnAUR9skqdHCK',
       '08mG3Y1vljYA6bvDt4Wqkj', '1jJci4qxiYcOHhQR247rEU',
       '0rTV5WefWd1J3OwIheTzxM', '2aoo2jlRnM3A0NyLQqMN2f',
       '6MWtB6iiXyIwun0YzU6DFP', '0I3q5fE6wg7LIfHGngUTnV'],
      dtype='object')

In [10]:
frequent_tracks = track_counts[track_counts > 70]

In [25]:
df_frequent = pd.DataFrame(frequent_tracks.index)
df_frequent.columns = ['track_id']

In [34]:
df_frequent = df_frequent.set_index(df_frequent.track_id)

In [38]:
df_frequent.rename(columns={'track_id': 'test_track_id'}, inplace=True)

In [73]:
all_tops = df_frequent.join(df_all_test[['track_name', 'track_id', 'artist_names']].set_index('track_id'), on = 'track_id')

In [84]:
df_all_working = pd.DataFrame(all_tops.apply(lambda x: pd.Series(x['artist_names']),axis=1).stack().reset_index(level=1, drop=True)).join(df_all_test[['track_name', 'track_id']].set_index('track_id'), on = 'track_id')


In [85]:
df_all_working.drop_duplicates()

,0,track_name
track_id,,
0bYg9bo50gSsH3LtXe2SQn,Mariah Carey,All I Want for Christmas Is You
2xLMifQCjDGFmkHkpNLD9h,Travis Scott,SICKO MODE
2JvzF1RMd7lE3KmFlsyZD8,J. Cole,MIDDLE CHILD
2374M0fQpWi3dLnB54qaLX,Toto,Africa
40riOy7x9W7GXjyGp4pjAv,Eagles,Hotel California - 2013 Remaster
7sO5G9EABYOXQKNPNiE9NR,Offset,Ric Flair Drip (& Metro Boomin)
7sO5G9EABYOXQKNPNiE9NR,Metro Boomin,Ric Flair Drip (& Metro Boomin)
7w87IxuO7BDcJ3YUqCyMTT,Foster The People,Pumped Up Kicks
2dpaYNEQHiRxtZbfNsse99,Marshmello,Happier


In [24]:
df_all_test

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta
10,brendan.ta,5cOhR878H8hC3UsxYq5Xyv,All My Friends (feat. Tinashe & Chance the Rap...,"[2FwJwEswyIUAljqgjNSHgP, 0NIIxcxNHmOoyBx03SfTC...","[Snakehips, Tinashe, Chance the Rapper]",2016-04-29T05:34:47Z,3pTPXB3vT93AOTSVozP54o,6TaqooOXAEcijL6G1AWS2K,All My Friends (feat. Tinashe & Chance the Rap...,brendan.ta
100,brendan.ta,07hs3meNvGu5Fp46pnLQm7,Hungry Ghost,[2N2EFVDEbp2JB8ulEUVIxp],[Violent Soho],2017-02-16T03:09:55Z,3pTPXB3vT93AOTSVozP54o,14NnWDVM8nCssfHAqWZp1T,Covered In Chrome,brendan.ta
1000,brendan.ta,4PKH86wn7Gw4iel2WD564k,Do It Like You,[0aA1GTrIMutjIh4GlPPUVN],[Crooked Colours],2019-02-04T05:22:30Z,3LF6iEL6uCE9rSdLF4m1ga,5h1Gi50T9QxVfzvIF8vciC,Do It Like You,brendan.ta
10000,crow74,4yR5YaZ4sKvDZOQ7CWVHZt,Epilogues for the End of the Sky,[3MZHEsxGNxPt6WIMwDCby2],[Bvdub],2018-06-11T18:34:47Z,6zqCzNUH0CjPiaRtMhY0WC,5iSxrPmZbINugN4ZsgUwwt,It All Ends with the Coming Sky,crow74
100000,theadamjanes,02azWnEq6UjQiR1gjEZ8ZC,Seen It All: The Autobiography (Deluxe),[4yBK75WVCQXej1p04GWqxH],[Jeezy],2019-05-03T13:39:22Z,3lmstM41QOd2Au4hZsE7ob,3ACwUaYu0EUjDCvu5xaEtG,Holy Ghost,theadamjanes
100001,theadamjanes,6jlrjFR9mJV3jd1IPSplXU,Thank Me Later,[3TVXtAsR1Inumwj472S9r4],[Drake],2019-05-03T13:39:22Z,3lmstM41QOd2Au4hZsE7ob,0llA0pYA6GpGk7fTjew0wO,The Resistance,theadamjanes
100002,theadamjanes,1WBZyULtlANBKed7Zf9cDP,Greatest Hits,[1ZwdS5xdxEREPySFridCfh],[2Pac],2019-05-03T13:39:22Z,3lmstM41QOd2Au4hZsE7ob,0sbhiygtRahuRj3DXPvtfe,Brenda's Got A Baby,theadamjanes
100003,theadamjanes,67ve7wATbOTIUMJi4dZA0s,Underground Underdog,[4nXOZlYoAD67hF9aUEncMY],[Pouya],2019-05-03T13:39:22Z,3lmstM41QOd2Au4hZsE7ob,5rFva7ZNcT61icb9dXavS1,41,theadamjanes
